In [1]:
import sys
sys.path.append("/scratch/group/csce435-f24/python-3.10.4/lib/python3.10/site-packages")
sys.path.append("/scratch/group/csce435-f24/thicket")
from glob import glob

import matplotlib.pyplot as plt
import pandas as pd

import thicket as th

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

Read all files

In [2]:
#1_trial is a name of a folder containing the cali files, you may create a folder with a different name and replace the folder name here
tk = th.Thicket.from_caliperreader(glob("cali_files/sample/*.cali"))

(1/2) Reading Files: 100%|██████████| 1/1 [00:00<00:00, 36.37it/s]
0it [00:00, ?it/s]
/home/alex/.local/lib/python3.10/site-packages/thicket/ensemble.py:409: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].replace({numerical_fill_value: None}, inplace=True)


View Calltree

In [3]:
print(tk.tree(metric_column="Avg time/rank"))

  _____ _     _      _        _   
 |_   _| |__ (_) ___| | _____| |_ 
   | | | '_ \| |/ __| |/ / _ \ __|
   | | | | | | | (__|   <  __/ |_ 
   |_| |_| |_|_|\___|_|\_\___|\__|  v2024.1.0

16.560 main
├─ 7.269 MPI_Comm_dup
├─ 0.054 MPI_Comm_split
├─ 0.000 MPI_Finalize
├─ 0.000 MPI_Finalized
├─ 0.000 MPI_Init
├─ 0.000 MPI_Initialized
├─ 1.605 comm
│  ├─ 0.392 comm_large
│  │  ├─ 0.074 MPI_Alltoall
│  │  ├─ 0.029 MPI_Alltoallv
│  │  ├─ 0.014 MPI_Gather
│  │  ├─ 0.168 MPI_Gatherv
│  │  └─ 0.108 MPI_Scatter
│  └─ 1.213 comm_small
│     └─ 1.213 MPI_Bcast
├─ 5.545 comp
│  ├─ 5.495 comp_large
│  └─ 0.050 comp_small
├─ 0.230 correctness_check
└─ 0.279 data_init_runtime
   └─ 0.123 MPI_Gather

Legend (Metric: Avg time/rank Min: 0.00 Max: 16.56 indices: {'profile': 1940458563})
█ 14.90 - 16.56
█ 11.59 - 14.90
█ 8.28 - 11.59
█ 4.97 - 8.28
█ 1.66 - 4.97
█ 0.00 - 1.66

name User code    ◀  Only in left graph    ▶  Only in right graph



Group Performance data by `matrix_size` in the Thicket metadata table.

In [ ]:
tk.metadata_column_to_perfdata("num_procs")
tk.metadata_column_to_perfdata("matrix_size")

tk.dataframe = tk.dataframe.reset_index().set_index(["node", "num_procs", "matrix_size"]).sort_index()

tk.dataframe.head()

In [ ]:
# Define common variables
processes = [2, 4, 8, 16, 32, 64]
matrix_sizes = [128, 1024, 8192]

In [ ]:
# Change font size for all plots
plt.rcParams.update({"font.size": 20})
print(tk.dataframe.columns)


In [ ]:
def plot_pandas(df, title, xlabel, ylabel):
    plot = df.plot(kind="line", 
        marker='o', 
        figsize=(15,7)
    )
    # Set title, xlabel, and ylabel explicitly
    plot.set_title(title)
    plot.set_xlabel(xlabel)
    plot.set_ylabel(ylabel)
    return plot


In [ ]:
# Saving plots for 'Total time'
for node in tk.graph.traverse():
    plot = plot_pandas(
        df=tk.dataframe.loc[node, "Total time"].unstack(level="matrix_size"),
        title=node.frame["name"] + " Total Computation Times",
        xlabel="Processors",
        ylabel="Time (seconds)",
    )
    # Save the plot with a unique filename for each node
    plt.savefig(f"{node.frame['name']}_total_time_plot.png")
    plt.close()  # Close the figure to avoid overlapping plots

# Saving plots for 'Min time/rank'
for node in tk.graph.traverse():
    plot = plot_pandas(
        df=tk.dataframe.loc[node, "Min time/rank"].unstack(level="matrix_size"),
        title=node.frame["name"] + " Minimum Times",
        xlabel="Processors",
        ylabel="Time (seconds)",
    )
    plt.savefig(f"{node.frame['name']}_min_time_plot.png")
    plt.close()

# Saving plots for 'Avg time/rank'
for node in tk.graph.traverse():
    plot = plot_pandas(
        df=tk.dataframe.loc[node, "Avg time/rank"].unstack(level="matrix_size"),
        title=node.frame["name"] + " Average Times",
        xlabel="Processors",
        ylabel="Time (seconds)",
    )
    plt.savefig(f"{node.frame['name']}_avg_time_plot.png")
    plt.close()

# Saving plots for 'Max time/rank'
for node in tk.graph.traverse():
    plot = plot_pandas(
        df=tk.dataframe.loc[node, "Max time/rank"].unstack(level="matrix_size"),
        title=node.frame["name"] + " Maximum Time",
        xlabel="Processors",
        ylabel="Time (seconds)",
    )
    plt.savefig(f"{node.frame['name']}_max_time_plot.png")
    plt.close()